**(Version 1)**
- Trying `ResNet50` 
- Epochs 60
- Datasize 5000
- Test data
- Keep last 10 layers trainable
- *Note:* Removed all layers after base_model except the necessary ones

**(Version 2)**
- Trying `InceptionV3`


### ResNet152V2

### DenseNet201 Training all

### V6: DenseNet201 Training all - Soft Tissues

### V7: DenseNet201 - L2 Regularization - alpha 0.01

### V8: DenseNet201 - L1 L2 Regularization - alpha 0.0001

### V9: DenseNet201 - L2 Regularization - alpha 0.001

### V11: On 30,000 images without Regularization

### V12: On 107,933 images without Regularization

### V13: All data on DenseNet121

### V14: All data on DenseNet201 but with batch size of 128 and 8 epochs - Failed

### V15: All data on DenseNet201 but with batch size of 64 and 8 epochs


In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #disable warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import seaborn as sns
import seaborn as sns
import pydicom
import math
import tensorflow_hub as hub
import matplotlib
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15) 
sns.set_style("darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 4})

In [ ]:
print(pd.__version__)
print(tf.__version__)
print(pydicom.__version__)
print(np.__version__)

In [ ]:
!cat /proc/cpuinfo

In [ ]:
def plot_learning_curves(history, metrics_to_plot = ['loss','binary_accuracy']):
  import math
  ncols = 2
  nrows = math.ceil(len(metrics_to_plot) / 2)
  if len(metrics_to_plot) <= 2:
        fig, axes = plt.subplots(nrows,ncols, figsize=(20,10))
        for i in range(2):
            axes[i].plot(history.history[metrics_to_plot[i]], label=metrics_to_plot[i] +' (training data)')
            axes[i].plot(history.history['val_'+metrics_to_plot[i]], label=metrics_to_plot[i] + ' (test data)')
            axes[i].set_ylabel('Value', fontsize = 20)
            axes[i].set_xlabel('No. epoch', fontsize = 20)
            axes[i].legend(prop={'size': 20})
            axes[i].set_title(metrics_to_plot[i], size = 22)
  else:        
      fig, axes = plt.subplots(nrows,ncols, figsize=(15,20))

      for i in range(ncols):
        for j in range(nrows):
          metric_idx = j * ncols + i
          if metric_idx >= len(metrics_to_plot):
                break
          axes[j,i].plot(history.history[metrics_to_plot[metric_idx]], label=metrics_to_plot[metric_idx] +' (training data)')
          axes[j,i].plot(history.history['val_'+metrics_to_plot[metric_idx]], label=metrics_to_plot[metric_idx] + ' (test data)')
          axes[j,i].set_ylabel('Value', fontsize = 20)
          axes[j,i].set_xlabel('No. epoch', fontsize = 20)
          axes[j,i].legend(prop={'size': 20})
          axes[j,i].set_title(metrics_to_plot[metric_idx], size = 22)

In [ ]:
SEED = 42
SAMPLE_NORMAL = 107933
SAMPLE_ABNORMAL = 107933
SUBCLASSES = ['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']
IMAGE_SIZE = (224,224)
BATCH_SIZE = 64
NUM_CLASSES = 1
EPOCHS = 8
ALPHA = 0.0001
METRICS = [tf.keras.metrics.BinaryAccuracy()]

In [ ]:
TRAIN_PATH = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/'
train_df = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv')
train_df.head()

In [ ]:
label = train_df.Label
train_df = train_df.ID.str.rsplit('_', n=1, expand=True)
train_df['label'] = label
train_df.rename({0: 'id', 1: 'subtype'}, axis=1, inplace = True)
train_df.head()

In [ ]:
train_df = pd.pivot_table(train_df, index='id', columns = 'subtype', values= 'label')
train_df.head()

In [ ]:
train_df.index = train_df.index.astype(str) + '.dcm'
train_df.head()

In [ ]:
train_df['any'].value_counts()

In [ ]:
train_df.head()

In [ ]:
train_df = train_df[['any']]
train_df.head()

In [ ]:
normal_df = train_df[train_df['any'] == 0]
abnormal_df = train_df[train_df['any'] == 1]

In [ ]:
normal_sample = normal_df.sample(SAMPLE_NORMAL, replace = False, random_state = SEED, axis = 0)
abnormal_sample = abnormal_df.sample(SAMPLE_ABNORMAL, replace = False, random_state = SEED, axis = 0)
sample_df = normal_sample.append(abnormal_sample)
sample_df = sample_df.sample(frac = 1, random_state = SEED, axis = 0)
sample_df.head()

In [ ]:
sample_df['any'].value_counts()

In [ ]:
sample_df['any']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sample_df.index, sample_df, test_size = 0.3, random_state = SEED)
X_train, X_val, y_train, y_val = train_test_split(y_train.index, y_train, test_size = 0.3, random_state = SEED)

In [ ]:
len(y_test), len(y_val), len(y_train)

In [ ]:
train_df = y_train
val_df = y_val
test_df = y_test

In [ ]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000
    
def get_first_of_dicom_field_as_int(x):
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    return int(x)
    
def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, # window center
                    data[('0028','1051')].value, # window width
                    data[('0028','1052')].value, # intercept
                    data[('0028','1053')].value, # slope
                   ]
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]
    

def get_min_max_of_window_value(window_center, window_width):
    mini = window_center - (window_width // 2)
    maxi = window_center + (window_width // 2) 
    return mini, maxi

def window_image(img, window_center, window_width):
    try:
        _,_, intercept, slope = get_windowing(img)
        img = img.pixel_array * slope + intercept
        img_min, img_max = get_min_max_of_window_value(window_center, window_width)
        img[img < img_min] = img_min
        img[img > img_max] = img_max
#         img = (img - np.min(img)) / (np.max(img) - np.min(img)) #normalize
    except:
        img = img_min * np.ones(IMAGE_SIZE)
        
    return img

def normalize(channel, wc_ww: tuple, norm_type = 'none'):
    if norm_type.lower() == 'none':
        return channel
    if norm_type.lower() == 'min_max':
        mini, maxi = get_min_max_of_window_value(wc_ww[0], wc_ww[1])
        resulted_channel = (channel - mini) / (maxi - mini)
        return resulted_channel
    

def bsb_window(img):
    bsb_config = {'brain': (40,80),
             'subdural': (80,200),
             'soft': (50, 350)}
    brain_img = window_image(img, *bsb_config['brain'])
    subdural_img = window_image(img,*bsb_config['subdural'])
    soft_img = window_image(img, *bsb_config['soft'])
    
    brain_img = normalize(brain_img, bsb_config['brain'], 'min_max')
    subdural_img = normalize(subdural_img, bsb_config['subdural'], 'min_max')
    soft_img = normalize(soft_img, bsb_config['soft'], 'min_max')
    
#         print(np.min(soft_img))
#     brain_img = (brain_img - 0) / 80
#     subdural_img = (subdural_img - (-20)) / 200
#     soft_img = (soft_img - (-150)) / 380 # (-150 = 40 - 380 / 2)
#         print(np.min(soft_img))
    bsb_img = np.zeros((brain_img.shape[0], brain_img.shape[1],3))
    bsb_img[:, :, 0] = brain_img
    bsb_img[:, :, 1] = subdural_img
    bsb_img[:, :, 2] = soft_img
    
    if (np.any(np.isnan(bsb_img))):
        bsb_img = np.ones((*IMAGE_SIZE,3))
        
    return bsb_img


    
class ImageGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe,batch_size,shuffle, num_classes = NUM_CLASSES):
        self.dataframe = dataframe
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.shuffle = shuffle
        
    def __len__(self):
        return math.ceil(len(self.dataframe) / self.batch_size)
    
    def __getitem__(self, index):
        batch_df = self.dataframe.iloc[index * self.batch_size: (index+1) * self.batch_size]
        paths = TRAIN_PATH + batch_df.index.astype(str)
        X = np.empty((len(batch_df), *IMAGE_SIZE, 3))
        y = np.empty((len(batch_df), self.num_classes))
        for i, path in enumerate(paths):
            dcm = pydicom.dcmread(path)
            # correct dcm
            if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
                correct_dcm(dcm)
#             rescaled_img = rescale_pixelarray(dcm)
#             windowed_img = set_manual_window(rescaled_img, HU_MIN, HU_MAX)
#             img = tf.convert_to_tensor(windowed_img, dtype=tf.float32)
            img = bsb_window(dcm)
            img = tf.convert_to_tensor(img, dtype=tf.float64)
#             assert tf.reduce_min(img) >= -1 and tf.reduce_max(img) <= 1, 'Check these values img in (-1,1)'
            X[i] = tf.image.resize(img, IMAGE_SIZE)
            y[i] = batch_df.iloc[i].values
#             assert tf.reduce_min(y[i]) >= 0 and tf.reduce_max(y[i]) <= 1, 'Check target values in (0,1)'
            
        return X, y
    
    def on_epoch_end(self):
        if self.shuffle:
            self.dataframe = self.dataframe.sample(len(self.dataframe), replace = False, random_state = SEED)
        self.current_epoch += 1

In [ ]:
import os
import tempfile

def add_regularization(model, regularizer=tf.keras.regularizers.l2(0.0001)):

    if not isinstance(regularizer, tf.keras.regularizers.Regularizer):
      print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
      return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
              setattr(layer, attr, regularizer)

    # When we change the layers attributes, the change only happens in the model config file
    model_json = model.to_json()

    # Save the weights before reloading the model.
    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    # load the model from the config
    model = tf.keras.models.model_from_json(model_json)
    
    # Reload the model weights
    model.load_weights(tmp_weights_path, by_name=True)
    return model

In [ ]:
img_generator_train = ImageGenerator(train_df, BATCH_SIZE, shuffle=True)
img_generator_val = ImageGenerator(val_df, BATCH_SIZE, shuffle = True)
img_generator_test = ImageGenerator(test_df, BATCH_SIZE, shuffle = False)
train_data = tf.data.Dataset.from_generator(lambda: map(tuple, img_generator_train), 
                                            output_types=(tf.float64, tf.uint8),
                                            output_shapes = (
                                                    tf.TensorShape((None, *IMAGE_SIZE,3)),
                                                    tf.TensorShape((None, NUM_CLASSES))
                                            ))
val_data = tf.data.Dataset.from_generator(lambda: map(tuple, img_generator_val), 
                                          output_types=(tf.float64, tf.uint8),
                                          output_shapes = (
                                                    tf.TensorShape((None, *IMAGE_SIZE,3)),
                                                    tf.TensorShape((None, NUM_CLASSES))
                                            ))
test_data = tf.data.Dataset.from_generator(lambda: map(tuple, img_generator_test), 
                                          output_types=(tf.float64, tf.uint8),
                                          output_shapes = (
                                                    tf.TensorShape((None, *IMAGE_SIZE,3)),
                                                    tf.TensorShape((None, NUM_CLASSES))
                                            ))



data_augmentation = tf.keras.Sequential([
   tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
   tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
   tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
   tf.keras.layers.experimental.preprocessing.RandomHeight(0.2),
   tf.keras.layers.experimental.preprocessing.RandomWidth(0.2)
])

# base_model = hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v1/feature_vector/5", trainable=False)
base_model = tf.keras.applications.DenseNet121(include_top = False)

for layer in base_model.layers:
    layer.trainable = True
# for layer in base_model.layers[-10:]:
#     layer.trainable = True
# base_model = add_regularization(base_model,regularizer=tf.keras.regularizers.l2(ALPHA))
    
# for layer in base_model.layers[:-10]:
#     layer.trainable = False

inputs = tf.keras.layers.Input(shape = IMAGE_SIZE + (3,), name = "input_layer")
x = data_augmentation(inputs)

x = base_model(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.8)(x)

outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer = tf.keras.optimizers.Adam(learning_rate=0.000125),
             metrics = METRICS)
print(model.summary())
history = model.fit(train_data, 
                   epochs = EPOCHS,
                   validation_data = val_data,
                   callbacks = [tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min', save_freq='epoch'),
                               tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=5),
                               tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.5,
                                           patience=2,
                                           min_lr=1e-8,
                                           mode="min")
                               ]
)

In [ ]:
model.evaluate(test_data)

In [ ]:
val_data

In [ ]:
pred_prob = model.predict(test_data)
pred_labels = (pred_prob > 0.5).astype(int)

In [ ]:
res = pd.DataFrame({'prob': pred_prob.flatten(), 'label': pred_labels.flatten()}, index = test_df.index)
comp = pd.DataFrame({'res_prob': res.prob, 'res_label': res.label, 'test_label': test_df['any']}, index = test_df.index)
comp.head(20)

In [ ]:
# Check the accuracy
(comp['res_label'] == comp['test_label']).sum() / comp.shape[0]

In [ ]:
test_df.head(10)

In [ ]:
res.to_csv('res.csv')
test_df.to_csv('test_df.csv')
comp.to_csv('comp.csv')

In [ ]:
plt.plot(history.history["lr"], 'o-')
plt.title('Learning Rate')
plt.xlabel('Epochs')
plt.ylabel('LR')
plt.show()

In [ ]:
plot_learning_curves(history)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(classification_report(test_df['any'], pred_labels))

In [ ]:

def plot_confusion_matrix(cm, class_name=None,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    # Plot confusion matrix in a beautiful manner
    fig = plt.figure(figsize=(12, 10))
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, fmt = 'g', cmap = cmap); #annot=True to annotate cells
    # labels, title and ticks
    ax.set_xlabel('Predicted', fontsize=20)
    ax.xaxis.set_label_position('bottom')
    plt.xticks(rotation=0)
    ax.xaxis.tick_bottom()

    ax.set_ylabel('True', fontsize=20)
    plt.yticks(rotation=0)

    plt.title(title, fontsize=20)
    plt.show()

In [ ]:
plot_confusion_matrix(confusion_matrix(test_df['any'], pred_labels))